In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor 
import time


In [2]:
df = pd.read_excel(r'C:\Users\Nafi Kareem\OneDrive\Dokumen\Data MIning\Penelitian\Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833.0,12,0.21,2,22.0,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800.0,12,0.21,2,22.0,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826.0,12,0.21,2,22.0,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831.0,12,0.21,2,22.0,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834.0,12,0.21,2,22.0,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676.0,36,0.21,2,41.0,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683.0,36,0.21,2,42.0,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340909,50151.250000,77962.397727,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846.0,36,0.21,2,42.0,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694118,47515.850980,73948.509804,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876.0,36,0.21,2,42.0,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943182,42468.990341,65994.485795,3.789568e+07,3761.343750


In [3]:
df.dropna(inplace=True)

In [4]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['NOx[g/s]']

In [5]:
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [7]:
mlp = MLPRegressor()
mlp

MLPRegressor()

In [8]:
start_time = time.time()
mlp.fit(X_train, y_train)
print("Execution time: " + str((time.time() - start_time)) + ' s')

Execution time: 20.036264896392822 s


In [9]:
y_pred = mlp.predict(X_test)

In [11]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 0.3522710617782455


In [12]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape:.2f}%")

MAPE: 2.39%


In [13]:
y_pred_inverse= TargetVarScalerFit.inverse_transform(y_pred.reshape(-1, 1))
y_test_inverse= TargetVarScalerFit.inverse_transform(y_test.reshape(-1, 1))
y_error = np.abs(y_pred_inverse - y_test_inverse)

mlp_table = pd.DataFrame({
    'Predicted NOx (g/s)': y_pred_inverse.ravel(),
    'Actual NOx (g/s)': y_test_inverse.ravel(),
    'Error': y_error.ravel()
})

print(mlp_table)

       Predicted NOx (g/s)  Actual NOx (g/s)     Error
0                 0.030158           0.03617  0.006012
1                 0.070084           0.06710  0.002984
2                 0.031516           0.02744  0.004076
3                 0.037159           0.04414  0.006981
4                 0.016327           0.01547  0.000857
...                    ...               ...       ...
11254             0.021087           0.02237  0.001283
11255             0.006551           0.00798  0.001429
11256             0.035193           0.04986  0.014667
11257             0.028019           0.02438  0.003639
11258             0.025030           0.02859  0.003560

[11259 rows x 3 columns]


In [14]:
mlp_table.to_excel(r'C:\Users\Nafi Kareem\OneDrive\Dokumen\Data MIning\Penelitian\prediction\mlp_prediction_results.xlsx', index=False)

Grid

In [18]:
param = {
    'hidden_layer_sizes':[100,150],
    'activation':['relu','tanh'],
    'alpha':[0.00001, 0.0001],
    'solver':['adam','sgd'],
    'learning_rate':['constant','adaptive'],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)


In [19]:
grid = GridSearchCV(mlp, 
                    param, 
                    cv=5, 
                    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
                    refit='RMSE',
                    n_jobs=-1,
                    verbose=0
                    )


start_time = time.time()
grid.fit(X_train, y_train)


print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.410361 using {'activation': 'tanh', 'alpha': 1e-05, 'hidden_layer_sizes': 150, 'learning_rate': 'adaptive', 'solver': 'sgd'}
Best RMSE: 0.41036112722815454
Best MAPE: -1.918645882497239
Execution time: 806.2959580421448 s


c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [20]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid:.2f}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.40
MAPE: 2.66%


Random

In [21]:
rs = RandomizedSearchCV(mlp, 
                            param, 
                            cv=5, 
                            scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
                            refit='RMSE',
                            n_iter=10,
                            verbose=0,
                            n_jobs=-1
                            )


start_time = time.time()
rs.fit(X_train, y_train)


print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.404582 using {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': 150, 'alpha': 0.0001, 'activation': 'tanh'}
Best RMSE: 0.4045821990549644
Best MAPE: -1.9185512558171598
Execution time: 258.8980944156647 s


c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs:.2f}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.39
MAPE: 2.55%


Bayessian

In [23]:
def mlp_bs(hidden_layer_sizes, activation, solver, alpha, learning_rate):
    activations = ['relu','tanh']
    solvers = ['adam','sgd']
    learning_rates = ['constant','adaptive']
    mlpbs= MLPRegressor(
        hidden_layer_sizes=int(hidden_layer_sizes),
        activation=activations[int(activation)],
        solver=solvers[int(solver)],
        alpha=float(alpha),
        learning_rate=learning_rates[int(learning_rate)],
    )
    scores = cross_val_score(mlpbs, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse 

In [24]:
bayes_opt = BayesianOptimization(
    mlp_bs,
    {
        'hidden_layer_sizes': (100, 150),
        'activation': (0, 1),
        'solver': (0, 1),
        'alpha': (0.00001, 0.0001),
        'learning_rate': (0, 1)
    }
)

In [25]:
start_time = time.time()
bayes_opt.maximize(init_points=5, n_iter=10)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | activa... |   alpha   | hidden... | learni... |  solver   |
-------------------------------------------------------------------------------------
| 1         | -0.3541   | 0.3094    | 4.446e-05 | 134.6     | 0.6445    | 0.1876    |
| 2         | -0.3542   | 0.315     | 3.874e-05 | 108.1     | 0.9345    | 0.9103    |
| 3         | -0.355    | 0.2273    | 9.9e-05   | 141.9     | 0.7946    | 0.8039    |
| 4         | -0.3556   | 0.1256    | 8.002e-05 | 118.1     | 0.3248    | 0.6891    |
| 5         | -0.3547   | 0.4017    | 7.043e-05 | 104.3     | 0.5912    | 0.5406    |
| 6         | -0.3534   | 0.387     | 3.606e-05 | 132.6     | 0.03156   | 0.1321    |


c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Nafi Kareem\anaconda3\Lib\site-packages

| 7         | -0.4063   | 1.0       | 4.286e-05 | 130.7     | 0.953     | 1.0       |
| 8         | -0.3534   | 0.5847    | 8.932e-05 | 133.1     | 0.203     | 0.139     |
| 9         | -0.3538   | 0.02575   | 3.417e-05 | 110.5     | 0.06825   | 0.1117    |
| 10        | -0.3527   | 0.9804    | 1.994e-05 | 112.8     | 0.716     | 0.9017    |
| 11        | -0.3561   | 0.1601    | 9.158e-05 | 115.0     | 0.1433    | 0.0981    |
| 12        | -0.353    | 0.2543    | 9.645e-05 | 138.7     | 0.8367    | 0.4598    |
| 13        | -0.3527   | 0.7583    | 9.739e-05 | 136.8     | 0.1375    | 0.9556    |
| 14        | -0.3545   | 0.9688    | 5.805e-05 | 101.2     | 0.3724    | 0.9243    |
| 15        | -0.3564   | 0.9298    | 7.965e-05 | 144.9     | 0.4516    | 0.172     |
Execution time: 834.3574287891388 s


In [26]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [27]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'activation': 1.0, 'alpha': 4.285549845802041e-05, 'hidden_layer_sizes': 130.65140370501197, 'learning_rate': 0.9529598468358368, 'solver': 1.0}


In [31]:
activations = ['relu','tanh']
solvers = ['adam','sgd']
learning_rates = ['constant','adaptive']

best_bs = MLPRegressor(
    hidden_layer_sizes=int(best_params['hidden_layer_sizes']),
    activation=activations[int(best_params['activation'])],  # Convert int to string
    solver=solvers[int(best_params['solver'])],  # Convert int to string
    alpha=float(best_params['alpha']),
    learning_rate=learning_rates[int(best_params['learning_rate'])]  # Convert int to string
)
best_bs.fit(X_train, y_train)

c:\Users\Nafi Kareem\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(activation='tanh', alpha=4.285549845802041e-05,
             hidden_layer_sizes=130, solver='sgd')

In [32]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs:.2f}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.40
MAPE: 2.53%
